# Importing necessary libraries

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC  
from sklearn.metrics import classification_report, confusion_matrix  
from sklearn import linear_model, decomposition, datasets
from sklearn import svm
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
import math
from mlxtend.plotting import plot_confusion_matrix

In [27]:
#retrieving the data
parkingdata = pd.read_csv("parking_duration_of_parking_event_vs_street_ID.csv")
#Selecting random sample size of 10000 rows of data from the large dataset
parkingdata = parkingdata.sample(n=10000)
parkingdata.size

130000

# Data Wrangling

In [28]:
#Converting all the feature attributes to uppercase for uniformity

parkingdata['Area Name'] = parkingdata['Area Name'].str.upper()
parkingdata['Street Name'] = parkingdata['Street Name'].str.upper()
parkingdata['Between Street 1'] = parkingdata['Between Street 1'].str.upper()
parkingdata['Between Street 2'] = parkingdata['Between Street 2'].str.upper()
parkingdata['Street Marker'] = parkingdata['Street Marker'].str.upper()
parkingdata['Sign'] = parkingdata['Sign'].str.upper()

parkingdata.head(5)

,Area Name,Street Name,Between Street 1,Between Street 2,Side Of Street,Street Marker,Arrival Time,Departure Time,Duration of Parking Event (in seconds),Sign,In Violation?,Street ID,Device ID
5031647,CITY SQUARE,FLINDERS LANE,SWANSTON STREET,RUSSELL STREET,4,1765S,24/02/2012 10:52:59 AM,24/02/2012 11:03:39 AM,640,P10 M-SUN 0:00-23:59,0,669,1432
10773353,THE MAC,LT LONSDALE STREET,SWANSTON STREET,RUSSELL STREET,3,3416N,17/08/2012 03:34:25 PM,17/08/2012 05:34:26 PM,7201,2P MTR M-SAT 7:30-20:30,0,926,5665
5924046,SOUTHBANK,KAVANAGH STREET,BALSTON STREET,POWER STREET,2,8714E,22/03/2012 08:18:17 AM,22/03/2012 10:37:39 AM,8362,3P TKT A M-F 7:30-18:30,0,828,4754
3003261,HYATT,EXHIBITION STREET,LT COLLINS STREET,COLLINS STREET,1,C380,22/12/2011 01:37:47 PM,22/12/2011 02:15:25 PM,2258,1P MTR M-SAT 7:30-19:30,0,647,275
4339221,HYATT,EXHIBITION STREET,LT COLLINS STREET,COLLINS STREET,1,C352,03/02/2012 12:35:41 PM,03/02/2012 12:38:58 PM,197,1P MTR M-SAT 7:30-19:30,0,647,4940


In [29]:
#Removing all redundant extra whitespaces
for x in parkingdata.columns:
    if parkingdata[x].dtype == object:
        parkingdata[x] = parkingdata[x].str.strip()

In [30]:
#converting object to datetime
parkingdata['Arrival Time'] = pd.to_datetime(parkingdata['Arrival Time'])
parkingdata['Departure Time'] = pd.to_datetime(parkingdata['Departure Time'])

In [31]:
#check which columns have missing values
parkingdata.columns[parkingdata.isnull().any()]

Index([], dtype='object')

# Target field identification and feature engineering

In [32]:
#Target field identification
target = parkingdata['Street Name']

unique_vals = target.unique()
target.replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

target.head(10)

5031647     0
10773353    1
5924046     2
3003261     3
4339221     3
11346637    1
1521817     4
8431751     5
977256      6
10683493    7
Name: Street Name, dtype: int64

In [33]:
#Preliminary Target Features identification (Pre-Hill Climbing)

#Convert possible targets to dtype = int
intDf = parkingdata.copy()
#Drop useless fields
intDf.drop('Arrival Time', axis = 1, inplace = True)
intDf.drop('Departure Time', axis = 1, inplace = True)
intDf.drop('Street Marker', axis = 1, inplace = True)
#Drop target field
intDf.drop('Duration of Parking Event (in seconds)', axis = 1, inplace = True)
#Drop already int fields (Will add back after)
intDf.drop('In Violation?', axis = 1, inplace = True)
intDf.drop('Side Of Street', axis = 1, inplace = True)
intDf.drop('Street ID', axis = 1, inplace = True)
intDf.drop('Device ID', axis = 1, inplace = True)

#Transform the remainder non-int fields' values to unique int identifiers
for column in intDf:
    unique_vals = intDf[column].unique()
    intDf[column].replace(to_replace = unique_vals, value = list(range(len(unique_vals))), inplace = True)

#Create dataframe for all the relevant features
features = intDf.copy()

#Add back int fields (If not the target field)
features['In Violation?'] = parkingdata['In Violation?']
features['Side Of Street'] = parkingdata['Side Of Street']
features['Street ID'] = parkingdata['Street ID']
features['Device ID'] = parkingdata['Device ID']
#Add back datetime features with only the meaningful subset of data
features['Arrival Hour'] = parkingdata['Arrival Time'].dt.hour
features['Departure Hour'] = parkingdata['Departure Time'].dt.hour

#Table view of features dataframe
features.head(10)

,Area Name,Street Name,Between Street 1,Between Street 2,Sign,In Violation?,Side Of Street,Street ID,Device ID,Arrival Hour,Departure Hour
5031647,0,0,0,0,0,0,4,669,1432,10,11
10773353,1,1,0,0,1,0,3,926,5665,15,17
5924046,2,2,1,1,2,0,2,828,4754,8,10
3003261,3,3,2,2,3,0,1,647,275,13,14
4339221,3,3,2,2,3,0,1,647,4940,12,12
11346637,4,1,3,3,1,0,4,926,1537,15,15
1521817,5,4,4,4,4,0,2,1171,3038,8,9
8431751,0,5,5,5,5,0,2,627,5180,12,12
977256,6,6,6,6,3,0,3,123,1379,7,8
10683493,7,7,7,7,6,0,2,641,7344,12,12


# Parameter tuning and Random Forest Data Modelling 

In [34]:
#Parameter Tuning

#Define the parameters to tune and the values to tune to
params_randomforest = [
                { 
                   'n_estimators' : (10,20,30,50,70,90,100),
                    'criterion' : ('gini', 'entropy'),
                    'max_depth' : (3, 5, 7, 9, 10, 15),
                    'max_features' : ('auto', 'sqrt'),
                    'min_samples_split' : (2, 4, 6, 8, 10, 14, 16)
                    
                }
            ]

In [35]:
cv_method = RepeatedStratifiedKFold(n_splits = 5, 
                                    n_repeats = 1, 
                                    random_state = 1)

gs_randomforest = GridSearchCV(RandomForestClassifier(), 
                      param_grid = params_randomforest, 
                      cv = cv_method,
                      verbose = True,
                      scoring = 'accuracy',
                      n_jobs = -1,
                      return_train_score = True)

In [36]:
#Fit the model with the dataset
bestModel = gs_randomforest.fit(features, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   25.1s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   56.9s
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 1226 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1776 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 2426 tasks      | elapsed:  5.5min
[Parallel(n_jobs=-1)]: Done 3176 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 4026 tasks      | elapsed:  9.2min
[Parallel(n_jobs=-1)]: Done 4976 tasks      | elapsed: 11.6min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 13

In [37]:
gs_randomforest.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 70}

In [38]:
#Find the best values for the  parameters of the model (Formatted output)
best_e = bestModel.best_estimator_.get_params()['n_estimators']
best_C = bestModel.best_estimator_.get_params()['criterion']
best_maxd =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf =  bestModel.best_estimator_.get_params()['max_features']
best_mins =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimator:', best_e)
print('Best criterion:', best_C)
print('Best max depth:', best_maxd)
print('Best max features:', best_maxf)
print('Best min_samples_split:', best_mins)

Best estimator: 70
Best criterion: gini
Best max depth: 15
Best max features: auto
Best min_samples_split: 2


In [39]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5602
1,gini,3,auto,2,20,0.5960
2,gini,3,auto,2,30,0.6039
3,gini,3,auto,2,50,0.6011
4,gini,3,auto,2,70,0.6109
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9998
1172,entropy,15,sqrt,16,50,0.9995
1173,entropy,15,sqrt,16,70,0.9998
1174,entropy,15,sqrt,16,90,0.9997


In [40]:
#Re-Define model with the optimal parameter values AFTER HILL CLIMBING
randomforest = metric = RandomForestClassifier(n_estimators = best_e, 
                               criterion = best_C, 
                               max_depth = best_maxd, 
                               max_features = best_maxf,
                               min_samples_split = best_mins                
                               )

In [41]:
# Hill climbing w/ Randomforest
new_Ind = []
cur_MaxScore = 0.0
col_num = len(features.columns)
col_Ind_Random = shuffle(range(0, col_num), random_state = 1)
features_array = features.values

for cur_f in range(col_num):
    new_Ind.append(col_Ind_Random[cur_f])
    newData = features_array[:, new_Ind]
    x_train, x_test, y_train, y_test = train_test_split(newData, target, test_size=0.2, random_state=1)
    fit = randomforest.fit(x_train, y_train)
    cur_Score = randomforest.score(x_test, y_test)
    
    if cur_Score < cur_MaxScore:
        new_Ind.remove(col_Ind_Random[cur_f])
    else:
        cur_MaxScore = cur_Score
        print ("Score with " + str(len(new_Ind)) + " selected features: " + str(cur_Score))
print("\nIndexs of the desired features")
print(new_Ind)

Score with 1 selected features: 0.3765
Score with 2 selected features: 0.4455
Score with 3 selected features: 0.7695
Score with 4 selected features: 0.9985
Score with 5 selected features: 0.9985
Score with 6 selected features: 1.0
Score with 7 selected features: 1.0
Score with 8 selected features: 1.0
Score with 9 selected features: 1.0

Indexs of the desired features
[2, 3, 4, 1, 0, 7, 10, 8, 5]


In [42]:
features_hc = pd.DataFrame()
for index in new_Ind:
    colName = features.columns[index]
    features_hc[colName] = features[colName]
features_hc.head(10)

,Between Street 1,Between Street 2,Sign,Street Name,Area Name,Street ID,Departure Hour,Device ID,In Violation?
5031647,0,0,0,0,0,669,11,1432,0
10773353,0,0,1,1,1,926,17,5665,0
5924046,1,1,2,2,2,828,10,4754,0
3003261,2,2,3,3,3,647,14,275,0
4339221,2,2,3,3,3,647,12,4940,0
11346637,3,3,1,1,4,926,15,1537,0
1521817,4,4,4,4,5,1171,9,3038,0
8431751,5,5,5,5,0,627,12,5180,0
977256,6,6,3,6,6,123,8,1379,0
10683493,7,7,6,7,7,641,12,7344,0


In [43]:
bestModel = gs_randomforest.fit(features_hc, target)

Fitting 5 folds for each of 1176 candidates, totalling 5880 fits


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done 212 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done 462 tasks      | elapsed:   59.6s
[Parallel(n_jobs=-1)]: Done 812 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 1262 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 1812 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 2462 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 3212 tasks      | elapsed:  7.1min
[Parallel(n_jobs=-1)]: Done 4062 tasks      | elapsed:  9.1min
[Parallel(n_jobs=-1)]: Done 5012 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 5880 out of 5880 | elapsed: 13

In [44]:
gs_randomforest.best_params_

{'criterion': 'gini',
 'max_depth': 15,
 'max_features': 'auto',
 'min_samples_split': 2,
 'n_estimators': 70}

In [45]:
best_e_hc = bestModel.best_estimator_.get_params()['n_estimators']
best_C_hc = bestModel.best_estimator_.get_params()['criterion']
best_maxd_hc =  bestModel.best_estimator_.get_params()['max_depth']
best_maxf_hc =  bestModel.best_estimator_.get_params()['max_features']
best_mins_hc =  bestModel.best_estimator_.get_params()['min_samples_split']

print('Best estimators:', best_e_hc)
print('Best criterion:', best_C_hc)
print('Best max depth:', best_maxd_hc)
print('Best max features:', best_maxf_hc)
print('Best min_samples_split:', best_mins_hc)

Best estimators: 70
Best criterion: gini
Best max depth: 15
Best max features: auto
Best min_samples_split: 2


In [46]:
#Visualise the parameter configurations from fitting the model with the dataset
results_randomforest = pd.DataFrame(gs_randomforest.cv_results_['params'])
results_randomforest['test_score'] = gs_randomforest.cv_results_['mean_test_score']
results_randomforest

,criterion,max_depth,max_features,min_samples_split,n_estimators,test_score
0,gini,3,auto,2,10,0.5576
1,gini,3,auto,2,20,0.5844
2,gini,3,auto,2,30,0.5987
3,gini,3,auto,2,50,0.6056
4,gini,3,auto,2,70,0.5999
...,...,...,...,...,...,...
1171,entropy,15,sqrt,16,30,0.9996
1172,entropy,15,sqrt,16,50,0.9997
1173,entropy,15,sqrt,16,70,0.9998
1174,entropy,15,sqrt,16,90,0.9998


In [47]:
randomforest = metric = RandomForestClassifier(n_estimators = best_e_hc, 
                               criterion = best_C_hc, 
                               max_depth = best_maxd_hc, 
                               max_features = best_maxf_hc,
                               min_samples_split = best_mins_hc                
                               )

# Classification report

In [48]:
#Defining training and testing groups
x_train, x_test, y_train, y_test = train_test_split(features_hc, target, test_size = 0.5, random_state = 4)

#Training the model previously defined
randomforest.fit(x_train, y_train)

#Obtaining and printing out results from the model (Confusion Matrix)
predicted = randomforest.predict(x_test)
cm = metrics.confusion_matrix(y_test,predicted)
print (cm)
print("\n Accuracy score: {:.5f}".format(randomforest.score(x_test, y_test)))
print (classification_report(y_test,predicted))

[[143   0   0 ...   0   0   0]
 [  0 167   0 ...   0   0   0]
 [  0   0  38 ...   0   0   0]
 ...
 [  0   0   0 ...   3   0   0]
 [  0   0   0 ...   0   1   0]
 [  0   0   0 ...   0   0   1]]

 Accuracy score: 1.00000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       143
           1       1.00      1.00      1.00       167
           2       1.00      1.00      1.00        38
           3       1.00      1.00      1.00       287
           4       1.00      1.00      1.00       348
           5       1.00      1.00      1.00       229
           6       1.00      1.00      1.00       228
           7       1.00      1.00      1.00       142
           8       1.00      1.00      1.00       267
           9       1.00      1.00      1.00       100
          10       1.00      1.00      1.00       286
          11       1.00      1.00      1.00        61
          12       1.00      1.00      1.00       231
          13       1.00  

# K-folds mean score

In [49]:
#Repeat with KFold - creating groups
kf = KFold(n_splits = 5, random_state = 4, shuffle = True)


#Repeat with KFold - Training model (previously defined) and obtaining its output
kFoldTotal = 0
for k, (train_index, test_index) in enumerate(kf.split(features_hc)):
    x_train, x_test = features_hc.iloc[train_index], features_hc.iloc[test_index]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]
    randomforest.fit(x_train, y_train)
    kFoldTotal += randomforest.score(x_test, y_test)
    print("[fold {0}] score: {1:.5f}".format(k, randomforest.score(x_test, y_test)))

#Printing out the results
roundedTotal = round(kFoldTotal/5, 5)
print("\nRandom Forest mean score [5 folds] = " + str(roundedTotal))

[fold 0] score: 1.00000
[fold 1] score: 1.00000
[fold 2] score: 1.00000
[fold 3] score: 1.00000
[fold 4] score: 0.99950

Random Forest mean score [5 folds] = 0.9999
